# Projet maison n° 3

In [5]:
# imports
import pandas as pd
import numpy as np
import os
import re

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python `df_names_us()` qui produit un unique DataFrame avec tous les fichiers en utilisant **pandas** (par ex. avec la fonction `pandas.concat()`), pas de bash :)

Ordre et noms des colonnes du DataFrame : "year", "name", "gender", "births"

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini dans les différents fichiers (voir la documentation du dataset).

In [6]:
files = os.listdir("/Users/dorianfitton/Documents/Cours Télécom/Kit BIG DATA/kit_bgd/names")
path = "/Users/dorianfitton/Documents/Cours Télécom/Kit BIG DATA/kit_bgd/names"


In [3]:
# names us
def df_names_us():

    df = pd.DataFrame()
    for filename in files:
        file = pd.read_csv(path + "/"+filename, names = ["name", "gender", "births"], encoding = 'unicode_escape')
        year = str(filename)[3:7]
        file['year']=year
        df = pd.concat([df, file], axis=0)
    first_column = df.pop('year')

    df.insert(0, 'year', first_column)
    
    df = df.sort_values(by=["year",'gender', 'births'], ascending = [True, True,False])
    df = df.dropna()
    df=df.reset_index()
    df = df.drop(labels='index', axis=1)
    #print(df.info())
    return df

In [4]:
%%time
df_names_us()


CPU times: user 3.59 s, sys: 1.06 s, total: 4.65 s
Wall time: 4.73 s


,year,name,gender,births
0,1880,Mary,F,7065.0
1,1880,Anna,F,2604.0
2,1880,Emma,F,2003.0
3,1880,Elizabeth,F,1939.0
4,1880,Minnie,F,1746.0
...,...,...,...,...
2052776,2021,Zyeire,M,5.0
2052777,2021,Zyel,M,5.0
2052778,2021,Zyian,M,5.0
2052779,2021,Zylar,M,5.0


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction `df_names_fr()` qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne "gender"
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [73]:
# names fr
def df_names_fr():
    df = pd.DataFrame()
    df = pd.read_csv('/Users/dorianfitton/Documents/Cours Télécom/Kit BIG DATA/kit_bgd/nat2021.csv', delimiter = ";",header=0, names = ['gender', 'name', 'year', 'births'], keep_default_na=False)
    mapping = {1:'M', 2:'F'}
    df['name'] = df['name'].str.lower()
    df['name'] = df['name'].str.title()
    df['gender'] = df['gender'].map(mapping)
    df = df.drop(df[df['name'].str.len()<2].index)
    #print(len(df))
    #print(df.loc[df['name'].str.contains('_', na=False)])
    #print(df.loc[df['name'].str.contains('_', na=False)].index)
    df = df.drop(df.loc[df['name'].str.contains('_', na=False)].index)
    #print(len(df))
    first_column = df.pop('year')
    second_column = df.pop('name')
    df.insert(0, 'year', first_column)
    df.insert(1, 'name', second_column)
    df['births'] = df['births'].astype(float)
    df = df.sort_values(by=["year", 'gender', 'births'], ascending = [True, True, False])

    df = df.drop(df[~df['year'].str.startswith('1') &~df['year'].str.startswith('2')].index)
    #print(df[df.isnull().any(axis=1)])
    df=df.dropna()
    df = df.reset_index()
    df = df.drop(labels=['index'], axis=1)
    #print(len(df))
    #print(df.loc[df.name.str.contains(r"-[a-z]+$")])
    return df

In [74]:
df = pd.DataFrame()
df = pd.read_csv('/Users/dorianfitton/Documents/Cours Télécom/Kit BIG DATA/kit_bgd/nat2021.csv', delimiter = ";",header=0, names = ['gender', 'name', 'year', 'births'])
df.loc[df.isnull().any(axis=1)]


,gender,name,year,births
579411,2,NaN,2003,3
579412,2,NaN,XXXX,29


In [75]:
df = df_names_fr()

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

Les données sont dans le fichier "Webstat_Export.csv".

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3. On retiendra uniquement les colonnes se terminant effectivement par un code ISO3 entre parenthèses. Par ex., "Dollar des Etats-Unis (USD)".

Implémenter une fonction `df_taux_change()` qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction `pd.to_datetime()` avec le format`"%d/%m/%Y"`) et doit être trié par ordre croissant. Les colonnes du DataFrame doivent correspondre aux devises sélectionnées.

In [254]:
# taux de change

def df_taux_change(devises):
    df = pd.read_csv('/Users/dorianfitton/Documents/Cours Télécom/Kit BIG DATA/Kit-Big-Data/session3/Webstat_Export.csv',delimiter =';')
    first_value = 5
    #df.drop(df.index[0:first_value],0,inplace=True)
    df.drop(df.head(first_value).index, inplace=True)
    df['Titre :'][:]= pd.to_datetime(df['Titre :'][:], format = '%d/%m/%Y')
    df = df.sort_values(by='Titre :')
    #df.rename(columns=lambda x: re.search(r'\((.*?)\)',x).group(1))
    df = df.rename(columns= lambda x: re.sub(r'(.*)\((.*)\)',r'\2', x))
    df = df.set_index('Titre :')
    df = df[devises][first_value:]
    #df = df.sort_values(by='Titre :')
    for y in df.columns:
        df[y] = [x.replace(',', '.') for x in df[y]]
    df = df.dropna(axis=1)
    df = df.drop(df.loc[df.isin(['-']).any(axis=1)].index)
    df = df.astype(np.float64)
    
    
    
    
    return df

In [185]:
#df = df.drop(df.loc[df.isin(['-']).any(axis=1)].index)

,AUD,CAD
Titre :,,
2022-09-20,"1,4893","1,3268"
2022-09-19,"1,495","1,3294"
2022-09-16,"1,4894","1,3226"
2022-09-15,"1,4853","1,3172"
2022-09-14,"1,4873","1,3177"
...,...,...
1999-01-08,"1,8406","1,7643"
1999-01-07,"1,8474","1,7602"
1999-01-06,"1,882","1,7711"


In [172]:
#[x.replace(',', '.') for x in df['AUD']]

['1.4893',
 '1.495',
 '-',
 '-',
 '1.4894',
 '1.4853',
 '1.4873',
 '1.4736',
 '1.4749',
 '-',
 '-',
 '1.4704',
 '1.4824',
 '1.4748',
 '1.4651',
 '1.4616',
 '-',
 '-',
 '1.4671',
 '1.4651',
 '1.4591',
 '1.4472',
 '1.4529',
 '-',
 '-',
 '1.4333',
 '1.4306',
 '1.4389',
 '1.4437',
 '1.4478',
 '-',
 '-',
 '1.4584',
 '1.4617',
 '1.4655',
 '1.4463',
 '1.4508',
 '-',
 '-',
 '1.4496',
 '1.4532',
 '1.4682',
 '1.4687',
 '1.4607',
 '-',
 '-',
 '1.4713',
 '1.4607',
 '1.4681',
 '1.4745',
 '1.4535',
 '-',
 '-',
 '1.4646',
 '1.4535',
 '1.462',
 '1.4605',
 '1.4707',
 '-',
 '-',
 '1.4677',
 '1.4848',
 '1.4767',
 '1.4869',
 '1.4839',
 '-',
 '-',
 '1.4886',
 '1.4893',
 '1.4802',
 '1.49',
 '1.491',
 '-',
 '-',
 '1.4871',
 '1.4883',
 '1.4961',
 '1.518',
 '1.5205',
 '-',
 '-',
 '1.5382',
 '1.5099',
 '1.5256',
 '1.521',
 '1.5278',
 '-',
 '-',
 '1.5248',
 '1.5212',
 '1.5254',
 '1.5177',
 '1.5061',
 '-',
 '-',
 '1.5039',
 '1.4939',
 '1.5051',
 '1.5174',
 '1.4998',
 '-',
 '-',
 '1.4845',
 '1.4985',
 '1.4917',
 '

### Tests

In [262]:
import unittest

class Session3Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 2052781)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 648330)#fail
        # index
        self.assertTrue(isinstance(df.index, pd.RangeIndex))
        # test names
        self.assertTrue(df.loc[df.name.str.contains(r"^[A-Z]+$")].empty)
        self.assertTrue(df.loc[df.name.str.contains(r"-[a-z]+$")].empty)#fail
        # test gender
        self.assertEqual(len(df), len(df.loc[df.gender=="F"]) + len(df.loc[df.gender=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(["CHF", "GBP", "USD"])
        # colonnes
        self.assertEqual(list(df.columns), ["CHF", "GBP", "USD"])
        # index
        self.assertTrue(isinstance(df.index, pd.DatetimeIndex))
        # index trié
        self.assertEqual(list(df.index.argsort()), list(range(len(df))))
        # types taux
        self.assertTrue((df.dtypes == float).all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [263]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session3Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [264]:
# run tests
run_tests()

test_df_names_fr (__main__.Session3Tests) ... ok
test_df_names_us (__main__.Session3Tests) ... ok
test_df_taux_change (__main__.Session3Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 6.328s

OK
